In [1]:
# This command can be used to install the required packages for this project.
# !pip install -r ../requirements.txt

In [2]:
# Add the parent directory of 'KernelCT' to sys.path

import sys
sys.path.append('..')

In [3]:
# Import necessary packages/modules
import numpy as np
from KernelCT.object import Object
from KernelCT import utilities, phantom
from KernelCT import kernel_reconstruction as kr
from KernelCT.weighted_kernels import WeightedGaussian

In [ ]:
# Setup

## Define phatom
phantom_type = 'shepp_logan'

## Generate pixel grid
x_max = 1
y_max = 1
grid_size = [256, 256]
X, Y = utilities.generate_pixel_grid(x_max, y_max, grid_size)
phantom_eval = phantom.eval_phantom(X, Y, phantom_type)

## Create object
geometry = 'random'
r_max = np.sqrt(2)
sample_size = 10000

r, a = utilities.generate_lineset(geometry, r_max, sample_size)
Radon = phantom.eval_phantom_radon(r, a, phantom_type)
obj = Object(r, a, Radon)

## Parameters for greedy selection
kernel = WeightedGaussian()
greedy_method = 'beta_greedy'
steps = 3
steps_size = 500
beta = 0.5
c_newton = None
V_newton = None
pwr_func_vals = None
res = None

In [ ]:
# Init figure of reconstructions

In [ ]:
# Run greedy kernel thinning
for i in range(steps):
    c_newton, V_newton, pwr_func_vals, res = obj.kernel_thinning(
        kernel, greedy_method, max_iter=steps_size, beta=beta,
        c_newton=c_newton, V_newton=V_newton, pwr_func_vals=pwr_func_vals,
        res=res
    )

    # plot reconstructions

beta_greedy: 100%|██████████████████████████████| 1500/1500 [00:33<00:00, 14.92it/s]


In [ ]:
# Modify plot + show